In [2]:
import sys, os, time
#%load_ext autoreload
#%autoreload 2
import torch
import torch.nn.functional as F
from torch.autograd import Variable, grad
import torch.distributions as distrib
import torch.nn as nn
import torch.utils.data as td
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
#%matplotlib notebook
%matplotlib tk

import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
from scipy.sparse import coo_matrix
#from envs import Discrete_Pos_Trap, Gym_Env, Print_Env
from torch.distributions.utils import lazy_property
import foundation as fd
from foundation.util import replicate, Cloner
import torch.multiprocessing as mp
import time

In [13]:
def mysubprocess(shared_dict):
    item = shared_dict['list_item'] = list()
    item.append('test')
    shared_dict['list_item'] = item
    a = torch.arange(10)
    print('sub: {}'.format(a.is_shared()))
    shared_dict['t'] = a
    a[2] = 5
    print('sub: {}'.format(a.is_shared()))
    print('subprocess:', shared_dict)
    time.sleep(1)
    print('subprocess:', shared_dict)

manager = mp.Manager()
shared_dict = manager.dict()
p = mp.Process(target=mysubprocess, args=(shared_dict,))
print(shared_dict)
p.start()
time.sleep(0.1)
print(shared_dict)
shared_dict['t'][4] = 10
p.join()
print('main process:', shared_dict)

{}
sub: False
sub: True
subprocess: {'list_item': ['test'], 't': tensor([ 0.,  1.,  5.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])}
{'list_item': ['test'], 't': tensor([ 0.,  1.,  5.,  3.,  4.,  5.,  6.,  7.,  8.,  9.])}
subprocess: {'list_item': ['test'], 't': tensor([  0.,   1.,   5.,   3.,  10.,   5.,   6.,   7.,   8.,   9.])}
main process: {'list_item': ['test'], 't': tensor([  0.,   1.,   5.,   3.,  10.,   5.,   6.,   7.,   8.,   9.])}


In [15]:
manager.Pool()

<PoolProxy object, typeid 'Pool' at 0x7f9d2865cf98>

In [2]:
class Test(object):
    def __init__(self, x=1):
        self.a = 2*x
        self.b = 'asdf'
        self.timeout = 1
    def c(self):
        return 5
    def d(self, x):
        return x + self.a
    def e(self, x, y):
        return x+y
    def __len__(self):
        return 100
    def __getitem__(self, item):
        return item + 1
    def __iter__(self):
        return iter(range(self.a))
    def __next__(self):
        return self.a+1
    def __add__(self, other):
        self.a += other
        return self.a
    def wait(self, tim=1):
        #print('starting {}'.format(self.a))
        time.sleep(tim)
        #print('{} done'.format(self.a))
    
t = Test()

In [3]:
start = time.time()
t.wait(0.1)
print('total {}'.format(time.time() - start))

total 0.10030627250671387


In [4]:
#m, r = replicate(Test, separate_args=True, unique_init_kwargs=[{'x':5}, {'x':3}, {'x':-4}, {'x':3}])

In [9]:
m, c = clone(Test, num_clones=3, num_workers=3)

In [10]:
start = time.time()
c.wait(0.1)
print('total {}'.format(time.time() - start))

total 0.10194635391235352


In [7]:
c.b

['asdf',
 'asdf',
 'asdf',
 'asdf',
 'asdf',
 'asdf',
 'asdf',
 'asdf',
 'asdf',
 'asdf']

In [13]:
r.e([1,2,3,4], [0,1,2,3])

[1, 3, 5, 7]

In [22]:
r.__len__()

[100, 100, 100, 100]

In [10]:
r.d(4)

[14, 10, -4, 10]

In [23]:
r + 3

[13, 9, -5, 9]

In [6]:
r.hello = 'world'

In [9]:
r.test

[AttributeError("'super' object has no attribute '__getattr__'"),
 AttributeError("'super' object has no attribute '__getattr__'")]

In [8]:
m[0].test = 'works'

In [12]:
m[0].__len__()

100

In [24]:
for i, j, k, l in r:
    print(i,j, k, l)

0 0 None 0
1 1 None 1
2 2 None 2
3 3 None 3
4 4 None 4
5 5 None 5
6 6 None 6
7 7 None 7
8 8 None 8
9 None None None
10 None None None
11 None None None
12 None None None


In [7]:
r[100]

collecting
collected: 101
collected: 101


[101, 101]

In [16]:
r[0].d(5)

first __class__
first __class__
first _make_getter
first _execute
first replicas
first in_queues
first out_queues
first timeout


15

In [17]:
r.timeout

first timeout


20

In [18]:
len(r)

first replicas


2

In [24]:
t.__dict__.keys()

dict_keys(['a', 'b'])

In [25]:
getattr(Test, 'd')(t, 6)

8

In [2]:
torch.__version__

'0.3.0.post4'

In [4]:
import torch
from torch.autograd import grad
import torch.distributions as distrib
def get_n():
    mu = torch.Tensor([1,2])#torch.randn(2)
    mu.requires_grad = True
    sigma = torch.Tensor([2,3])#torch.rand(2) * 10 + 1
    sigma.requires_grad = True

    n = distrib.MultivariateNormal(loc=mu, covariance_matrix=torch.diag(sigma))
    return n
def check(fn):
    try:
        fn()
    except Exception as e:
        return 'failed - {}'.format(e)
    return 'passed'
def test1():
    n = get_n()
    s = torch.ones(2)
    grad(n.log_prob(s).sum(), n.covariance_matrix)#, retain_graph=True))
def test2():
    n = get_n()
    grad(n.log_prob(n.sample()).sum(), n.covariance_matrix)
def test3():
    n = get_n()
    s = torch.ones(2)
    grad(n.log_prob(s).sum(), n.covariance_matrix, retain_graph=True)
    grad(n.log_prob(n.sample()).sum(), n.covariance_matrix, retain_graph=True)
def test4():
    n = get_n()
    grad(n.log_prob(n.sample()).sum(), n.covariance_matrix, retain_graph=True)
print(check(test1))
print(check(test2))
print(check(test3))
print(check(test4))

passed
passed
passed
passed


In [3]:
mp.freeze_support()
mp.set_start_method('spawn')

In [4]:
def play(iq, oq, private, i=None):
    print('test')
    m = None
    while True:
        print('prepping')
        r = iq.get()
        print('got {}'.format(r))
        if r is None:
            print('killing w{}'.format(i))
            break
        if r[0] == 'init':
            m = r[1]
            print(m.net.bias.storage().data_ptr(), m.net.bias.data[0])
            m.net.bias.data[0] = i
            print(m.net.bias.storage().data_ptr(), m.net.bias.data[0])
        elif r[0] == 'a':
            oq.put(m.a(*r[1:]))
        elif r[0] == 'b':
            oq.put(m.b(*r[1:]))
        elif r[0] == 'c':
            oq.put(m.c(*r[1:]))
        elif r[0] == 'p':
            oq.put(private)
        else:
            print('w{} died: {}'.format(i,r))
            break
        #print('w{}'.format(i))
        
class Model(object):
    def __init__(self, n,m):
        self.shape = n,m
        self.net = nn.Linear(n,m)
    def cuda(self):
        self.net.cuda()
    def cpu(self):
        self.net.cpu()
    def copy(self):
        return Model(*self.shape)
    def a(self, x):
        return self.net(x)
    def b(self, x):
        return self.net.bias[0] + x
    def c(self, x):
        self.net.bias[0] += x
        return self.net.bias

In [5]:
start = time.time()
#m = Model(200000,1000)
m = Model(3,10)
#m.cuda()
a = m.net
print(time.time() - start)

0.0008473396301269531


In [6]:
m.__getattribute__('a')

<bound method Model.a of <__main__.Model object at 0x7fb6da847668>>

In [7]:
print(a.bias.storage().data_ptr(), a.bias[0], a.bias.storage().is_shared())

93921918596608 tensor(0.2623) False


In [29]:
iq = mp.Queue()
oq = mp.Queue()
num = 3
l = nn.Linear(3,4)
workers = [mp.Process(target=play, args=(iq,oq,l,i)) for i in range(num)]
for w in workers:
    w.daemon = True
    w.start()

test
prepping
test
prepping
test
prepping
got None
got None
got None
killing w2
killing w1
killing w0


In [38]:
workers[0].is_alive()

False

In [37]:
mp.active_children()

[<Process(Process-6, started daemon)>,
 <Process(Process-1, started daemon)>,
 <Process(Process-4, started daemon)>,
 <Process(Process-3, started daemon)>,
 <Process(Process-2, started daemon)>,
 <Process(Process-5, started daemon)>]

In [32]:
t = list(l.parameters())[0]
t

Parameter containing:
tensor([[-0.2374, -0.4831,  0.1673],
        [-0.4746,  0.1704,  0.3808],
        [ 0.5721, -0.4562,  0.1819],
        [-0.0998,  0.2670,  0.2239]])

In [24]:
t.is_shared()

False

In [25]:
ms = [m.copy() for _ in range(num)]
ms = [m]*num

In [26]:
# dispatch
N = num
start = time.time()
for i in range(N):
    iq.put(('init', ms[i]))
    time.sleep(0.001) # allow time for tensors to copy
print(time.time() - start)

0.018649816513061523


In [27]:
print(a.bias.storage().data_ptr(), a.bias[0], a.bias.storage().is_shared())

140425204940800 tensor(1.) True


In [28]:
print(a.bias.storage().data_ptr(), a.bias[0], a.bias.storage().is_shared())

140425204940800 tensor(1.) True


In [141]:
for o in output:
    print(o.storage().data_ptr(), o[0], o.storage().is_shared())

140076987572224 tensor(47.) True
140076987572224 tensor(47.) True
140076987572224 tensor(47.) True
140076987572224 tensor(47.) True
140076987572224 tensor(47.) True
140076987572224 tensor(47.) True
140076987572224 tensor(47.) True
140076987572224 tensor(47.) True
140076987572224 tensor(47.) True
140076987572224 tensor(47.) True


In [36]:
for _ in workers:
    iq.put(None)
for w in workers:
    w.join()

In [19]:
#start = time.time()
#result = pool.map(lambda x: play(a,x), range(10))
#print(time.time() - start)

In [ ]:
env = fd.util.get_env('pendulum')

In [ ]:
pool = mp.Pool(4)

In [ ]:
N = 4
pool.map(lambda x: play(*x), zip([env]*N))

In [5]:
data = np.stack(data[3:])
data.shape

(9, 111, 2)